<a href="https://colab.research.google.com/github/Nischal2015/ncit-workshop/blob/main/1-basic_agents/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 2
### Building a "Financial Analyst" Agent

### Only run this if you are using Google Colab

In [ ]:
!pip install langsmith["openai-agents"] openai-agents langchain langchain-openai tavily-python

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "ncit-workshop"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

### Imports

In [ ]:
from agents import Agent, Runner, FunctionTool, function_tool, set_trace_processors
from langsmith.wrappers import OpenAIAgentsTracingProcessor
import nest_asyncio

nest_asyncio.apply()
set_trace_processors([OpenAIAgentsTracingProcessor(name="Financial Agents Tracer")])

### Defining Tools

Tools provides Agents (LLM) with the following capabilities:
- Interact with external systems, access real-time information, and perform actions beyond text generation.
- Call APIs, execute code, retrieve data, and automate complex workflows.
- Extend their functionality beyond reasoning and language generation.

In [ ]:
@function_tool
def calculate_mortgage(principal: float, rate: float, years: int) -> float:
    """Calculates the monthly mortgage payment.

    Args:
        principal (float): The loan principal amount.
        rate (float): The annual interest rate (in decimals).
        years (int): The loan term in years.

    Returns:
        float: The monthly mortgage payment rounded to two decimal places.
    """
    monthly_rate = rate / 100 / 12
    payments = years * 12
    if monthly_rate == 0:
        return principal / payments
    payment = (principal * monthly_rate) / (1 - (1 + monthly_rate) ** -payments)
    return round(payment, 2)


@function_tool
def get_interest_rates(region: str) -> str:
    """Returns the current mortgage interest rates for a given region.

    Args:
        region (str): The region to get interest rates for.

    Returns:
        str: A string describing the current interest rates.
    """
    rates = {
        "US": "The current average mortgage interest rate in the US is around 3.5% for a 30-year fixed loan.",
        "Nepal": "The current average mortgage interest rate in Nepal is around 9% for a 20-year fixed loan.",
    }
    return rates.get(region, "Interest rate information not available for this region.")

### Defining Financial Analyst Agent

In [ ]:
analyst_agent = Agent(
    name="Mortgage Analyst Agent",
    model="gpt-4.1-mini",
    instructions=(
        "You are a helpful finanical analyst agent that helps users with mortgage calculations and interest rate information."
    ),
    tools=[calculate_mortgage, get_interest_rates],
)

In [ ]:
import json

for tool in analyst_agent.tools:
    if isinstance(tool, FunctionTool):
        print(f"Tool Name: {tool.name}")
        print(f"Description: {tool.description}")
        print(f"Json schema: {json.dumps(tool.params_json_schema, indent=2)}\n")

In [ ]:
result = await Runner.run(
    starting_agent=analyst_agent,
    input="What would my payment be for a $500k house in the US for 30 years?",
)
print(result.final_output)

### Using external API as tool

In [ ]:
import os
from tavily import TavilyClient
from dotenv import load_dotenv

load_dotenv()

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [ ]:
@function_tool
def tavily_search(query: str) -> str:
    """
    Perform a web search using Tavily and return a summarized result.
    """
    response = tavily_client.search(query, search_depth="fast", max_results=3)
    results = response.get("results", [])
    return results or "No results found."